In [ ]:
!pip install datasets transformers evaluate sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00


In [ ]:
import re
import json

# Specify the path to your text file
text_file_path = "/content/pytorch.txt"

# Read the text file
with open(text_file_path, "r") as file:
    text = file.read()

# Split the text into code and explanation sections
sections = re.split("\[(code|explanation)\]", text)
sections = [section.strip() for section in sections if section.strip()]

# Create a list of dictionaries for each code-explanation pair
data = []
i = 0
while i < len(sections):
    if sections[i] == 'code':
        code = sections[i + 1]
        if i + 2 < len(sections) and sections[i + 2] == 'explanation':
            explanation = sections[i + 3]
            data.append({"code": code, "explanation": explanation})
            i += 4
        else:
            data.append({"code": code, "explanation": ""})
            i += 2
    else:
        i += 1

# Specify the file path for the JSON file
file_path = 'data.json'

# Save the data as JSON
with open(file_path, 'w') as json_file:
    json.dump(data, json_file)

print(f'Data saved to {file_path} as JSON.')


Data saved to data.json as JSON.


In [ ]:
from datasets import Dataset, DatasetDict

with open('data.json', 'r') as json_file:
    data1 = json.load(json_file)

In [ ]:
import pandas as pd
df = pd.DataFrame(data1)
df.head()

,code,explanation
0,"x = torch.zeros(1, requires_grad=True)\nwith t...",This code creates a tensor x with shape (1) fi...
1,is_train = False\nwith torch.set_grad_enabled(...,This code snippet demonstrates the usage of `t...
2,torch.set_grad_enabled(True)\ny = x * 2\ny.req...,"In this code snippet, `torch.set_grad_enabled(..."
3,torch.set_grad_enabled(False)\ny = x * 2\ny.re...,"Here, `torch.set_grad_enabled(False)` disables..."
4,"x = torch.tensor([1, 2, 3])\ntorch.is_tensor(x)",This code snippet creates a tensor x using the...


In [ ]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['code', 'explanation'],
    num_rows: 335
})

In [ ]:
from transformers import RobertaTokenizer

model_nm = 'Salesforce/codet5-base'
tokenizer = RobertaTokenizer.from_pretrained(model_nm)

In [ ]:
max_input_length = 512
max_target_length = 256

#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def preprocess(x):
  model_inputs = tokenizer(
      x['code'],
      max_length = max_input_length,
      padding=True, truncation=True, return_tensors='pt'
  )
  labels = tokenizer(
      x['explanation'],
      max_length = max_target_length,
      padding=True,truncation=True,return_tensors='pt'
  )
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

tok_ds = dataset.map(preprocess,batched=True)
tok_ds = tok_ds.remove_columns(['code','explanation'])

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [ ]:
split_ds = tok_ds.train_test_split(0.2,seed=42)
split_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 268
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 67
    })
})

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq
import torch
from torch.utils.data import DataLoader

model = T5ForConditionalGeneration.from_pretrained(model_nm)

In [ ]:
batch_size = 2
split_ds.set_format('torch')

data_collator = DataCollatorForSeq2Seq(model=model,tokenizer=tokenizer,return_tensors='pt')

train_dataloader = DataLoader(split_ds['train'],shuffle=True,
                              collate_fn=data_collator,batch_size=batch_size)

eval_dataloader = DataLoader(split_ds['test'],collate_fn=data_collator,
                             batch_size=batch_size)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),lr=2e-5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
pip install rouge_score -q

  Preparing metadata (setup.py) ... done


In [ ]:
from rouge_score import rouge_scorer
import numpy as np
from tqdm.auto import tqdm

def compute_rouge_score(predictions,references):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'],
                                    use_stemmer=True)
  scores = [scorer.score(prediction, reference) for prediction, reference in zip(predictions, references)]

  return scores

In [ ]:
num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs*num_update_steps_per_epoch

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=1,gamma=0.1)

In [ ]:
from torch.cuda.amp import GradScaler, autocast

progress_bar = tqdm(range(num_training_steps))
scaler = GradScaler()

for epoch in range(num_training_steps):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        with autocast():
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'],
                            labels=batch['labels'])
            logits = outputs.logits

            # Calculate loss
            loss = outputs.loss

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    rouge_scores = []
    for batch in eval_dataloader:
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            with autocast():
                outputs = model.generate(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    max_length=264
                )
        # Decode generated explanations and references
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

        # Compute rouge scores
        scores = compute_rouge_score(decoded_preds, decoded_refs)
        rouge_scores.extend(scores)

    # Calculate average ROUGE scores
    avg_rouge_scores = {
        'rouge1': sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores),
        'rouge2': sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores),
        'rougeL': sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores),
    }
    #print("Average ROUGE scores:", avg_rouge_scores)

    # Adjust learning rate
    lr_scheduler.step()
print("Average ROUGE scores:", avg_rouge_scores)


  0%|          | 0/134 [00:00<?, ?it/s]

In [ ]:
# Set the model to evaluation mode
model.eval()

# Provide your prompt as input
prompt = "rnn = nn.RNN(10, 20, 2)"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, truncation=True, return_tensors="pt").to(device)

# Generate output with explanation
with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        max_length=100,
        num_return_sequences=1,  # Set to generate a single explanation
        decoder_start_token_id=model.config.pad_token_id,  # Use the model's pad token as the decoder start token
        do_sample=True,  # Enable sampling for generating diverse explanations
        top_k=50,  # Set the top-k value for sampling
        temperature=0.8  # Adjust the temperature value for controlling randomness
    )

# Decode the generated output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated output
print(decoded_output)
